In [0]:

CREATE or REPLACE MATERIALIZED VIEW bolt_finint_int.silver.fs_max_transactions as

with global_payment_entities as (

-- Comment out the below once we switch to the raw catalogs
select record.transaction.*, 'AMER' as region
from bolt_dcp_prod.bronze.raw_s2s_payment_transaction_entities

UNION all

select 
record.transaction.*, 'LATAM' as region
from bolt_dcp_prod.any_latam_bronze.raw_s2s_payment_transaction_entities

UNION all 

select record.transaction.*, 'EMEA' as region
from bolt_dcp_prod.any_emea_bronze.raw_s2s_payment_transaction_entities

UNION all

select record.transaction.*, 'APAC' as region
from bolt_raw_prod.commerce.raw_any_apac_s2s_payment_transaction_entities

----------------------------------------------------------------------

-- Comment out below to switch ALL regions to the raw catalogs
-- UNION ALL

-- select tenant, record.transaction.*, 'AMER' as region
-- from bolt_raw_prod.commerce.raw_any_amer_s2s_payment_transaction_entities

-- UNION all

-- select tenant, record.transaction.*, 'LATAM' as region
-- from bolt_raw_prod.commerce.raw_any_latam_s2s_payment_transaction_entities

-- UNION all 

-- select tenant, record.transaction.*, 'EMEA' as region
-- from bolt_raw_prod.commerce.raw_any_emea_s2s_payment_transaction_entities

)
,

exploded_entities as (

select

case
  when st.event.refunded.providerRefundReference is not null then st.event.refunded.refundReference
  when st.event.chargeback.providerChargebackReference is not null then st.event.chargeback.providerChargebackReference
  else id
end as event_id,

case
  when st.event.refunding.providerRefundReference is not null then id
  when st.event.refunded.providerRefundReference is not null then id
  when st.event.chargeback.providerChargebackReference is not null then id
end as original_transaction_id,

realm,
region,
userId,
created as created_date,
st.event.occurred as event_occurred_date,
source.type as source_type,
source.reference as source_reference,

--populates the correct amount value depending on the event type
case
  when st.event.refunding.source is not null then 
    coalesce(
            st.event.refunding.amountDetails.amountWithTax.minorAmountInclTax * -1, 
            amountDetails.amountWithTax.minorAmountInclTax *-1,
            amountDetails.simpleAmount.minorAmount *-1
            )
  when st.event.refunded.providerRefundReference is not null then 
      coalesce(
            st.event.refunded.amountDetails.amountWithTax.minorAmountInclTax * -1, 
            amountDetails.amountWithTax.minorAmountInclTax *-1,
            amountDetails.simpleAmount.minorAmount *-1
            )
  when st.event.chargeback.providerChargebackReference is not null then 
      coalesce(
            st.event.chargeback.amountDetails.amountWithTax.minorAmountInclTax * -1, 
            amountDetails.amountWithTax.minorAmountInclTax *-1,
            amountDetails.simpleAmount.minorAmount *-1
            )
  else     coalesce(
            amountDetails.amountWithTax.minorAmountInclTax,
            amountDetails.simpleAmount.minorAmount
            )
end as billed_amount,

case
  when st.event.refunding.source is not null then 
    coalesce(
            st.event.refunding.amountDetails.amountWithTax.taxMinorAmount * -1, 
            amountDetails.amountwithTax.taxMinorAmount *-1
            )
  when st.event.refunded.providerRefundReference is not null then 
    coalesce(
            st.event.refunded.amountDetails.amountWithTax.taxMinorAmount * -1, 
            amountDetails.amountwithTax.taxMinorAmount *-1
            )
  when st.event.chargeback.providerChargebackReference is not null then 
    coalesce(
            st.event.chargeback.amountDetails.amountWithTax.taxMinorAmount * -1, 
            amountDetails.amountwithTax.taxMinorAmount *-1
            )
  else     coalesce(
            amountDetails.amountwithTax.taxMinorAmount,0)
end as Tax_amount,

amountDetails.currencyCode as currencyCode,
amountDetails.amountWithTax.taxRateMinorUnits/10000 as tax_rate,
amountdetails.amountWithTax.components[0].description as tax_description,
-- amountDetails.amountwithTax.taxRateMinorUnits as tax_rate_minor_units,
UPPER(amountDetails.amountwithTax.taxationCountryCode) as taxation_country_code,
-- amountDetails.amountwithTax.taxDocumentReference as tax_document_reference,

-- Determines the type of payment event
-- CREATED default means the payment StateTransitions was empty, likely meaning the transaction was created and nothing else yet (first step).
case
  when st.event.pending.providerPaymentReference is not null then 'PENDING'
  when st.event.successful.providerPaymentReference is not null then 'SUCCESSFUL'
  when st.event.canceled.reason is not null then 'CANCELLED'
  when (st.event.failed.providerPaymentReference is not null OR st.event.failed.reason is not null) then 'FAILED'
  when (st.event.retrying.nextretry is not null OR st.event.retrying.reason is not null) then 'RETRYING'
  when st.event.refunding.source is not null then 'REFUNDING'
  when st.event.refunded.providerRefundReference is not null then 'REFUNDED'
  when st.event.refundFailed.reason is not null then 'REFUND FAILED'
  when st.event.chargeback.providerChargebackReference is not null then 'CHARGEBACK'
  when st.event.timedOut.emptyObject is not null then 'TIMED OUT'
  when st.event.revoked.reason is not null then 'REVOKED'
  when st.event.chargebackRejected.providerChargebackReference is not null then 'CHARGEBACK REJECTED'
  when (st.event.corrected.providerPaymentReference is not null OR st.event.corrected.providerRefundReference is not null) then 'CORRECTED'
  else 'CREATED'
end as event_type,

case
  when st.event.pending.providerPaymentReference is not null then st.event.pending.providerPaymentReference
  when st.event.successful.providerPaymentReference is not null then st.event.successful.providerPaymentReference
  when st.event.failed.providerPaymentReference is not null then st.event.failed.providerPaymentReference
  when st.event.refunding.providerRefundReference is not null then st.event.refunding.providerRefundReference
  when st.event.refunded.providerRefundReference is not null then st.event.refunded.providerRefundReference
  when st.event.chargeback.providerChargebackReference is not null then st.event.chargeback.providerChargebackReference
  when st.event.chargebackRejected.providerChargebackReference is not null then st.event.chargebackRejected.providerChargebackReference
  when st.event.corrected.providerPaymentReference is not null then st.event.corrected.providerPaymentReference
  when st.event.corrected.providerRefundReference is not null then st.event.corrected.providerRefundReference
end as provider_Reference_id,

case
  when st.event.canceled.reason is not null then st.event.canceled.reason
  when st.event.failed.reason is not null then st.event.failed.reason 
  when st.event.retrying.reason is not null then st.event.retrying.reason
  when st.event.refundFailed.reason is not null then st.event.refundFailed.reason
  when st.event.revoked.reason is not null then st.event.revoked.reason
  when st.event.chargebackRejected.reason is not null then st.event.chargebackRejected.reason
  when st.event.timedOut.emptyObject is not null then st.event.timedOut.emptyObject
end as event_state_reason,

case
  when st.event.refunding.providerRefundReference is not null then st.event.refunding.source
  when st.event.refunded.providerRefundReference is not null then st.event.refunded.source
end as refund_source,

replace(provider,'PAYMENT_PROVIDER_','') as payment_provider,
replace(type, 'PAYMENT_TRANSACTION_TYPE_','') as payment_type,
paymentMethodId,
legacyExportingSource,
installmentDetails.numberOfInstallments as paymentInstallments,
merchantAccount,
items[0].subscriptionDetails.serviceperiod.startdate as service_period_startdate,
items[0].subscriptionDetails.serviceperiod.enddate as service_period_enddate,
st.event.retrying.nextretry as next_retry_date

from global_payment_entities
LATERAL VIEW explode_outer(stateTransitions) st as event -- this explodes out every state in the StateTranstions struct - should mirror events
)


select

---MAIN PAYMENT ENTITIY FIELDS---
t.realm,
t.region,
t.created_date,
t.event_occurred_date as transaction_date,

case 
  when t.service_period_startdate is null then t.created_date
  else t.service_period_startdate
end as invoice_start_date,

case 
  when t.service_period_enddate is null and p.paymentPeriod = 'PERIOD_MONTH' then add_months(t.created_date, p.numberOfPeriodsBetweenPayments)
  when t.service_period_enddate is null and p.paymentPeriod = 'PERIOD_YEAR' then add_months(t.created_date, 12)
  when t.service_period_enddate is null and p.paymentPeriod = 'PERIOD_DAY' then date_add(t.created_date, 1)
  when t.service_period_enddate is null and p.paymentPeriod = 'PERIOD_WEEK' then date_add(t.created_date, 7)
  else t.service_period_enddate
end as invoice_end_date,

t.next_retry_date,
t.source_type,
t.currencyCode as billed_currency,

t.event_type,
t.payment_type,
t.event_state_reason,
t.refund_source,
t.payment_provider,
t.paymentInstallments,
case 
    when t.region = 'EMEA' AND ((p.launch_wave IS NULL and t.event_occurred_date >= '2024-05-20T23:00:00.000+00:00') 
                            OR (p.launch_wave IS NOT NULL AND t.event_occurred_date >= '2024-06-11T00:00:00.000+00:00'))
    then TRUE
    when t.region = 'LATAM' AND t.event_occurred_date >= '2024-02-27T00:00:00.000+00:00' then TRUE
    when t.region = 'AMER' AND t.event_occurred_date >= '2023-05-23T00:00:00.000+00:00' then TRUE
    when t.region = 'APAC' AND t.event_occurred_date >= '2024-11-18T00:00:00.000+00:00' then TRUE
    else null
end as is_post_launch,



---PAYMENT METHOD ENTITIY INFO---
pm.paymentMethod_type as payment_method_type,
pm.cardDetails_cardProvider as card_Provider,
pm.cardDetails_fundingSource as funding_source,

---PLANS PRODUCTs CAMPAGINS ENTITIY INFO---
replace(p.market,'TG_MARKET_','') as priceplan_market,
p.launch_wave,
replace(p.producttype,'PRODUCT_TYPE_','') as product_type,
p.bundle as is_bundle,
p.retentionoffer as is_retention_offer,
p.productname,
replace(p.tiertype,'TIER_TYPE_','') as tier_type,
p.campaign_Name,
p.campaign_type,
p.internalname,
replace(p.priceplantype,'TYPE_','') as priceplantype,
replace(p.paymentPeriod,'PERIOD_','') as paymentperiod,
p.numberOfPeriodsBetweenPayments as period_frequency,
p.numberOfInstalments as PriceplanInstallments,
p.planprice as plan_price,
p.currency as plan_currency,
p.currencyDecimalPoints,

---SUBSCRIPTION ENTITY INFO---
s.origin as sub_origin,
s.purchaseTerritory as sub_purchase_territory,
s.direct_affiliate as sub_affiliate,
startDate as sub_start_date,
affiliateStartDate as sub_affiliate_start_date,
s.installmentType as sub_installment_type,

---AMOUNTS & CHARGE INFO---
round(coalesce(t.billed_amount, 0) / POWER(10, coalesce(p.currencyDecimalPoints,0)),2) as billed_amount,
round(coalesce(t.tax_amount, 0) / POWER(10, coalesce(p.currencyDecimalPoints,0)),2) as tax_amount,
round((coalesce(t.billed_amount, 0) - coalesce(t.tax_amount, 0)) / POWER(10, coalesce(p.currencyDecimalPoints,0)),2) as revenue_amount,
t.tax_rate,
t.tax_description,
UPPER(t.taxation_country_code) as tax_country_code,

---ALL REFERENCE IDs---
t.userid,
t.event_id,
t.original_transaction_id,
t.provider_Reference_id,
t.merchantaccount,
t.paymentmethodid,
t.legacyExportingSource,
s.globalsubscriptionid,
s.basesubscriptionid,
s.previoussubscriptionglobalid,
s.priceplanid,
p.businessType,
p.businessPlatform as SAP_Platform,
p.businessBrand as SAP_Brand,
p.revenueType as SAP_RevenueType,
p.businessCase,

---OTHER USEFUL ADHOC INFO
s.testUSer,
s.userEmail

from exploded_entities t
left join bolt_finint_int.silver.fs_max_subscriptions s on t.source_reference = s.globalsubscriptionid
left join bolt_finint_int.silver.fs_max_plans p on p.priceplanid = s.priceplanid
left join bolt_finint_int.silver.fs_paymentmethods pm on t.paymentmethodid = pm.paymentMethodId
